In [52]:
##set up
import numpy as np
import pandas as pd
import string
import email
import os ##use this to traverse folders
from email.parser import Parser
import re
import lxml.html
import collections

In [53]:
#extract labels from TREC2007/full/ file lists
with open('trec07p/full/index', 'r') as ifile:
    raw_labels = ifile.readlines()

#using file list to give labels and e-mail numbers

categories = [i.split( )[0] for i in raw_labels]
email_nums = [i.split(".")[-1][:-1] for i in raw_labels]
labels = pd.Series(categories, index=email_nums)

In [131]:
#reference from other place

# parser = Parser()
# corpus = []
# for email_num in email_nums:
#     filepath = 'trec07p/data/inmail.%s' % email_num

#     with open(filepath, 'r',) as ifile:
#         email = parser.parse(ifile)

#     ## TODO also keep the subject header

#     ## code and details from:
#     ## https://stackoverflow.com/questions/17874360/python-how-to-parse-the-body-from-a-raw-email-given-that-raw-email-does-not/32840516#32840516
#     if email.is_multipart():
#         for part in email.walk():
#             content_type = part.get_content_type()
#             content_dispo = str(part.get('Content-Disposition'))

#             # skip any text/plain (txt) attachments
#             if content_type == 'text/plain' and 'attachment' not in content_dispo:
#                 body = part.get_payload(decode=False)
#                 break ## only keep the first email
#     else:
#         body = email.get_payload(decode=False)

#     ## TODO strip html stuff ? or valuable features ?
#     try:
#         corpus.append((email_num, body))
#     except NameError:
#         pass

# email_nums, corpus = zip(*corpus)

In [18]:
# extract the body of email
def walkMsg(msg):
    """
    Input: string of email
    Output: body of email
    """
    for part in msg.walk():
        if part.get_content_type() == "multipart/alternative":
            continue
        yield part.get_payload(decode=1)

In [222]:
##encode
def robust_decode(bs):
    '''Takes a byte string as param and convert it into a unicode one.
First tries UTF8, and fallback to Latin1 if it fails'''
    cr = None
    try:
        #cr = bs.decode('utf8')
        cr = bs.decode('utf-8-sig')
    except UnicodeDecodeError:
        cr = bs.decode('ISO-8859-1')
        cr = cr.encode("ascii", "ignore")
        #cr = cr.decode('utf-8-sig')
    return cr
#.replace("\n"," ").replace("\t"," ").replace("r"," ")

In [244]:
def info_of_email_2():
    """
    Input: path, list of files name, classification(spam or ham)
    Ourput: a list with info of email
    """
    
    parser = Parser()
    info = []
    for nums in range(len(categories)):
        filepath = 'trec07p/data/inmail.%s' % email_nums[nums]
        
        with open(filepath, 'r') as ifile:
            email = parser.parse(ifile)
            
            Date = email.get("Date")

            Content = " ".join([robust_decode(i) for i in walkMsg(email) if i != None])
            Content = ''.join(Content.splitlines())
            Content = Content.replace("\t","")
            
            if 'html' in Content:
                Content = lxml.html.fromstring(Content).text_content()
            try:
                Date = str.split(Date)
                Day = Date[0].replace(',','')
                Hour = Date[4].split(':')[0]
                info.append([Date[3], Date[2], Date[1],Day, Hour,
                             email.get("From"),email.get('To'),
                             email.get('cc'),email.get('bcc'), 
                             str(email.get("Content-Type")).split(";")[0],
                             email.get('subject'),
                             Content,
                             categories[nums]])
            except:
                info.append([[], [], [], [], [],
                             email.get("From"), email.get('To'),
                             email.get('cc'), email.get('bcc'), 
                             str(email.get("Content-Type")).split(";")[0],
                             email.get('subject'),
                             Content,
                             categories[nums]])
    return info

In [239]:
df = pd.DataFrame(info_of_email_2(),
                 columns=['Year','Month','Day','Weekday',"Hour",
                                'From','To','cc','bcc','Content_type','Subjects','Content','Classification'])

In [131]:
##need to figure out the way to deal with wrong year/mo/day....##
collections.Counter(df.iloc[[len(i)==4 for i in df['Year']]]['Year'])

Counter({'1900': 1,
         '1970': 3,
         '1980': 6,
         '1987': 8,
         '1988': 2,
         '1990': 39,
         '1994': 2,
         '1997': 2,
         '1998': 10,
         '1999': 11,
         '2000': 46,
         '2001': 100,
         '2002': 144,
         '2003': 272,
         '2004': 186,
         '2005': 106,
         '2006': 133,
         '2007': 72543,
         '2008': 23,
         '2009': 5,
         '2010': 1,
         '2011': 1,
         '2012': 1,
         '2013': 1,
         '2015': 3,
         '2017': 3,
         '2020': 1,
         '2025': 1,
         '2027': 2,
         '2029': 1,
         '2031': 1,
         '2036': 1,
         '2038': 6,
         '2040': 1,
         '2080': 1,
         '2087': 2,
         '2090': 4,
         '2098': 1,
         '2099': 2,
         '2102': 2,
         '4307': 1,
         '9744': 1})

In [247]:
df.iloc[[len(i)!=4 for i in df['Year']]]

,Year,Month,Day,Weekday,Hour,From,To,cc,bcc,Content_type,Subjects,Content,Classification
48,18:28:59,2007,Apr,8,+0000,"""AccuWeather.com Alert"" <inbox@messaging.accuw...",<ktwarwic@speedy.uwaterloo.ca>,None,None,text/html,AccuWeather.com Alert(tm) Forecast For User|AV...,Alert ...,ham
49,18:28:59,2007,Apr,8,+0000,"""AccuWeather.com Alert"" <inbox@messaging.accuw...",<ktwarwic@speedy.uwaterloo.ca>,None,None,text/html,AccuWeather.com Alert(tm) Forecast For User|AV...,Alert ...,ham
315,13:25:51,2007,Apr,7,+0200,"""Mr.Eiji Cheng"" <eijicheng6@yahoo.com.hk>",smile@speedy.uwaterloo.ca,None,None,text/plain,please reply (investment trade),"DBS BANK.DBS Bank, Hong Kong Branch.16th Floor...",spam
323,13:39:59,2007,Apr,7,+0200,"""Mr.Eiji Cheng"" <eijicheng6@yahoo.com.hk>",smile@speedy.uwaterloo.ca,None,None,text/plain,please reply (investment trade),"DBS BANK.DBS Bank, Hong Kong Branch.16th Floor...",spam
392,04:27:49,2007,Apr,9,+0000,"""AccuWeather.com Alert"" <inbox@messaging.accuw...",<ktwarwic@speedy.uwaterloo.ca>,None,None,text/html,AccuWeather.com Alert(tm) Forecast For User|AV...,Alert ...,ham
393,04:27:49,2007,Apr,9,+0000,"""AccuWeather.com Alert"" <inbox@messaging.accuw...",<ktwarwic@speedy.uwaterloo.ca>,None,None,text/html,AccuWeather.com Alert(tm) Forecast For User|AV...,Alert ...,ham
516,04:53:26,09,Apr,Mon,2007,Webshield.SMTP.V4.5.MR2.Mail.Service@speedy.uw...,<mack@speedy.uwaterloo.ca>,None,None,None,Returned Mail: Error During Delivery,---- Failed Recipients ----Requested action no...,spam
608,07:22:34,2007,Apr,9,-0400,""" Mystery Shopping Rewards"" <MysteryShoppingRe...",<gnitpick@speedy.uwaterloo.ca>,None,None,text/html,Become a mystery shopper. Keep your purchases!,We'll give you a $500 Gift Card to your favori...,spam
659,12:35:29,2007,Apr,9,+0000,"""AccuWeather.com Alert"" <inbox@messaging.accuw...",<ktwarwic@speedy.uwaterloo.ca>,None,None,text/html,AccuWeather.com Alert(tm) Forecast For User|AV...,Alert ...,ham
660,12:35:29,2007,Apr,9,+0000,"""AccuWeather.com Alert"" <inbox@messaging.accuw...",<ktwarwic@speedy.uwaterloo.ca>,None,None,text/html,AccuWeather.com Alert(tm) Forecast For User|AV...,Alert ...,ham


In [361]:
filepath = 'trec07p/data/inmail.49'

with open(filepath, 'r') as ifile:
    email = parser.parse(ifile)

In [362]:
Date = email.get('Date')

In [363]:
print email

From inbox@messaging.accuweather.com  Sun Apr  8 14:29:23 2007
Return-Path: <inbox@messaging.accuweather.com>
Received: from messaging.accuweather.com (messaging.accuweather.com
 [207.242.93.76])
 by speedy.uwaterloo.ca (8.12.8/8.12.5) with ESMTP id l38ITM0I003872
 for <ktwarwic@speedy.uwaterloo.ca>; Sun, 8 Apr 2007 14:29:22 -0400
Received: from mail pickup service by messaging.accuweather.com with Microsoft
 SMTPSVC; Sun, 8 Apr 2007 18:28:59 +0000
To: <ktwarwic@speedy.uwaterloo.ca>
From: "AccuWeather.com Alert" <inbox@messaging.accuweather.com>
Reply-To: InBoxHelp@accuwx.com
Error-To: directwx@accuwx.com
Subject: AccuWeather.com Alert(tm) Forecast For User|AVCAVC
MIME-version: 1.0
Content-type: text/html; charset=us-ascii
Content-Disposition: inline; filename="accuwx.htm"
Content-Base: "http://www.accuweather.com"
Message-ID: <DIRECTWXd5i3h1WxIie000106ce@messaging.accuweather.com>
X-OriginalArrivalTime: 08 Apr 2007 18:28:59.0490 (UTC)
 FILETIME=[C6622C20:01C77A0B]
Date: 8 Apr 2007 18:

In [364]:
Date = [i.replace(",","") for i in Date.split()]

In [384]:
Weekday = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
t = [i in Weekday for i in Date]
Wd =[]
if len(t)==1:
    Wd = Date[[i for i, x in enumerate(t) if x][0]]

Yr = []
t = [len(i)==4 for i in Date]
if sum(t)==1:
    Yr = Date[[i for i, x in enumerate(t) if x][0]]

TypeError: list indices must be integers, not list